In [1]:
# Dependencies
import pandas as pd
import numpy as np
import pymongo
import time
import datetime as dt
import json
import geojson

In [2]:
#open geojson file
with open("../static/data/us_states.geojson", encoding="ISO-8859-1") as fp:
    geo = geojson.load(fp)
    

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.redfin
result = collection.find({'region_type': 'county', 'period_begin': "2020-12-28"})

df = pd.DataFrame(list(result))
sel_cols = [
"region_name" ,
"median_sale_price",
"median_sale_ppsf",
"inventory"
]
rf_df = df[sel_cols]

In [4]:
states = []
for region in rf_df["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
rf_df["state"] = states
rf_df.head()

C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,region_name,median_sale_price,median_sale_ppsf,inventory,state
0,"Iowa County, WI",133000.0,89.28571428571429,47.0,WI
1,"Shoshone County, ID",240000.0,98.07619766125987,21.0,ID
2,"Washoe County, NV",403997.5,242.17854384812048,959.0,NV
3,"Washtenaw County, MI",285090.0,163.96946564885496,767.0,MI
4,"Indiana County, PA",125000.0,106.56822082379864,161.0,PA


In [20]:
rf_df["median_sale_price"] = pd.to_numeric(rf_df["median_sale_price"])

rf_df["median_sale_ppsf"] = pd.to_numeric(rf_df["median_sale_ppsf"])

rf_df["inventory"] = pd.to_numeric(rf_df["inventory"])


C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is tryi

In [15]:
# agg_df = rf_df.groupby("state").agg(["mean"])
agg_df = rf_df.groupby("state").agg({"median_sale_price": "median", "median_sale_ppsf": "median", "inventory": "sum"})
agg_df.reset_index(inplace = True)
agg_df.head()

,state,median_sale_price,median_sale_ppsf,inventory
0,AK,297752.3,166.906638,3334.0
1,AL,184653.0,103.432912,15803.0
2,AR,144000.0,89.964808,7957.0
3,AZ,344750.0,174.015895,37458.0
4,CA,574437.5,304.371932,126802.0


In [16]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

# Simple test examples
if __name__ == '__main__':
    print("Wisconin --> WI?", us_state_abbrev['Wisconsin'] == 'WI')
    print("WI --> Wisconin?", abbrev_us_state['WI'] == 'Wisconsin')
    print("Number of entries (50 states, DC, 5 Territories) == 56? ", 56 == len(us_state_abbrev))

Wisconin --> WI? True
WI --> Wisconin? True
Number of entries (50 states, DC, 5 Territories) == 56?  True


In [17]:
new_geo = geo
for i in range(len(geo.features)):
    NAME = geo.features[i].properties["name"]
    for row in agg_df.iterrows():
        if us_state_abbrev[NAME] == row[1][0]:
            new_geo.features[i].properties["median_sale_price"] = round(row[1][1],2)
            new_geo.features[i].properties["median_sale_ppsf"] = round(row[1][2],2)
            new_geo.features[i].properties["inventory"] = round(row[1][3],2)         

In [18]:
new_geo.features[0].properties

{'name': 'Alabama',
 'density': 94.65,
 'median_sale_price': 184653.0,
 'median_sale_ppsf': 103.43,
 'inventory': 15803.0}

In [19]:
# #establish MongoDB connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db

# #load in totals
db.redfinmap.drop()
db.redfinmap.find()

db.redfinmap.insert_one(new_geo)